# Importing libirires shuffle, and hyperparameters [Cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html)

In [8]:
import numpy as np
import pandas as pd
import os
import sys
import glob
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import svm, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score,ShuffleSplit,LeaveOneOut
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc,classification_report
from matplotlib import pyplot as plt
from collections import Counter
from scipy.stats import norm, zscore
#import seaborn as sns; sns.set(font_scale=1.2)
from sklearn.externals import joblib
%matplotlib inline

# Load the dataset

In [11]:
path='/home/sultan/EEG/CLUSTER_channels/S_E12/allindnoiseindBF.xlsx'
dataset=pd.read_excel(path,header=None)
#dataset =pd.read_csv(path) # This is the csv read
dataset.shape

(32, 21)

In [24]:
#dataset

# Leaveout on Whole data

In [20]:
# X=dataset.iloc[:,16:21].values;# theta:1-6, alpha=6-11, beta=11-16: gamma=16-21 
# X_scaled = preprocessing.scale(X); # normalizing 0 mean and 1 stdv
# y=dataset.iloc[:,0].values
leaveout = LeaveOneOut()
#leaveout.get_n_splits(X)
# parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,.002,0.003]}
# svc = svm.SVC()
### Here is the leaveout
cn=0
pos=[1,6,11,16]; # This is the cluster number lhpos=[3,8,13,19],j=i+2 rhpos=[1,6,11,16], j=i+2
a=['Theta',"alpha",'beta', 'gamma']
for i in pos:
    j=i+5
    X=dataset.iloc[:,i:j].values;# theta:1-6, alpha=6-11, beta=11-16: gamma=16-21 # 1-3 RH, 3-5 LH
    #print X
    X_scaled = preprocessing.scale(X)
    y=dataset.iloc[:,0].values
    count = 0
    totalacc = 0
    cper=[];ytt=[];
    for train_index, test_index in leaveout.split(X_scaled):
        #print train_index,test_index
        ##X_trn, X_tst = X[train_index], X[test_index]
        X_trn, X_tst = X_scaled[train_index], X_scaled[test_index]
        y_trn, y_tst = y[train_index], y[test_index]
        #parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,.002,0.003]}
        parameters = {'kernel':['rbf'], 'C':[1, 10,100,1000],'gamma':[0.01,.002, 0.003]}
        #parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':np.logspace(-2, 3, 10),'gamma':np.logspace(-5, 4, 10)}
        svc = svm.SVC()
        clf = GridSearchCV(svc, parameters)
        #print X_trn, y_trn
        clf.fit(X_trn, y_trn)
        #print clf.best_estimator_
        acc = accuracy_score(clf.best_estimator_.predict(X_tst),y_tst)
        totalacc = totalacc + acc
        pred=clf.best_estimator_.predict(X_tst)
        cper.append(pred)
        ytt.append(y_tst)
        count = count + 1
        #print count, acc, totalacc; # See the how many correct over the whole
    score=totalacc/(count-1)*100.0
    #print 'Accuracy of',a[cn],'=', totalacc/count 
    #print("Total score for %s is %2.2f."%(a[cn], score));# see 2 digit results 
    print("Total accuracy score for {} is {}".format(a[cn], round(score,2)))
    #print (classification_report(np.array(ytt), np.array(cper)))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(np.array(ytt),np.array(cper))
    roc_acu=roc_auc = auc(false_positive_rate, true_positive_rate)
    print("ROC_AUC: {} is {}".format(a[cn], round(roc_acu,2)))
#     print '\n'
    #print('Acc: {}'.format(totalacc/count) 
    cn+=1 

Total accuracy score for Theta is 38.71
ROC_AUC: Theta is 0.32
Total accuracy score for alpha is 38.71
ROC_AUC: alpha is 0.32
Total accuracy score for beta is 48.39
ROC_AUC: beta is 0.39
Total accuracy score for gamma is 61.29
ROC_AUC: gamma is 0.57


In [ ]:
#parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,.002,0.003]}
# Total score for Theta is 50.0
# Total score for alpha is 34.375
# Total score for beta is 53.125
# Total score for gamma is 31.25
#### parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':np.logspace(-2, 3, 10),'gamma':np.logspace(-5, 4, 10)}
# Total score for Theta is 53.125
# Total score for alpha is 43.75
# Total score for beta is 37.5
# Total score for gamma is 43.75

# LH

In [19]:
# X=dataset.iloc[:,16:21].values;# theta:1-6, alpha=6-11, beta=11-16: gamma=16-21 
# X_scaled = preprocessing.scale(X); # normalizing 0 mean and 1 stdv
# y=dataset.iloc[:,0].values
leaveout = LeaveOneOut()
#leaveout.get_n_splits(X)
# parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,.002,0.003]}
# svc = svm.SVC()
### Here is the leaveout
cn=0
pos=[3,8,13,18]; # This is the cluster number lhpos=[3,8,13,18],j=i+2 rhpos=[1,6,11,16], j=i+2
a=['Theta',"alpha",'beta', 'gamma']
for i in pos:
    j=i+2
    X=dataset.iloc[:,i:j].values;# theta:1-6, alpha=6-11, beta=11-16: gamma=16-21 # 1-3 RH, 3-5 LH
    #print X
    X_scaled = preprocessing.scale(X)
    y=dataset.iloc[:,0].values
    count = 0
    totalacc = 0
    cper=[];ytt=[];
    for train_index, test_index in leaveout.split(X_scaled):
        #print train_index,test_index
        X_trn, X_tst = X_scaled[train_index], X_scaled[test_index]
        y_trn, y_tst = y[train_index], y[test_index]
        #parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,.002,0.003]}
        parameters = {'kernel':['rbf'], 'C':[1, 10,100,1000],'gamma':[0.01,.002, 0.003]}
        #parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':np.logspace(-2, 3, 10),'gamma':np.logspace(-5, 4, 10)}
        svc = svm.SVC()
        clf = GridSearchCV(svc, parameters)
        clf.fit(X_trn, y_trn)
        #print clf.best_estimator_
        acc = accuracy_score(clf.best_estimator_.predict(X_tst),y_tst)
        totalacc = totalacc + acc
        pred=clf.best_estimator_.predict(X_tst)
        cper.append(pred)
        ytt.append(y_tst)
        count = count + 1 
        #print count, acc, totalacc; # See the how many correct over the whole
    score=totalacc/(count-1)*100.0
    #print 'Accuracy of',a[cn],'=', totalacc/count
    print("Total accuracy score for {} is {}".format(a[cn], round(score,2)))
    #print (classification_report(np.array(ytt), np.array(cper)))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(np.array(ytt),np.array(cper))
    roc_acu=roc_auc = auc(false_positive_rate, true_positive_rate)
    print("ROC_AUC: {} is {}".format(a[cn], round(roc_acu,2)))
    #print '\n'
    cn+=1 

Total accuracy score for Theta is 45.16
ROC_AUC: Theta is 0.37
Total accuracy score for alpha is 45.16
ROC_AUC: alpha is 0.37
Total accuracy score for beta is 45.16
ROC_AUC: beta is 0.37
Total accuracy score for gamma is 80.65
ROC_AUC: gamma is 0.76


In [ ]:
# Total score for Theta is 59.375
# Total score for alpha is 59.375
# Total score for beta is 59.375
# Total score for gamma is 53.125

# RH

In [23]:
leaveout = LeaveOneOut()
#leaveout.get_n_splits(X)
#parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,0.02,0.05,0.08,.002,0.003]}
parameters = {'kernel':['rbf'], 'C':[1, 10,100,1000],'gamma':[0.01,.002, 0.003]}
#parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':np.linspace(0.1,2,10)}
#parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':np.logspace(-2, 3, 5),'gamma':np.logspace(-5, 4, 5)}
svc = svm.SVC()
#### Here is the leaveout
cn=0
pos=[1,6,11,16]; # This is the cluster number lhpos=[3,8,13,18],j=i+2 rhpos=[1,6,11,16], j=i+2
a=['Theta',"alpha",'beta', 'gamma']
for i in pos:
    j=i+2
    X=dataset.iloc[:,i:j].values;# theta:1-6, alpha=6-11, beta=11-16: gamma=16-21 # 1-3 RH, 3-5 LH
    X_scaled = preprocessing.scale(X)
    y=dataset.iloc[:,0].values
    count = 0
    totalacc = 0
    cper=[];ytt=[];
    for train_index, test_index in leaveout.split(X_scaled):
        #print train_index,test_index
        X_trn, X_tst = X_scaled[train_index], X_scaled[test_index]
        y_trn, y_tst = y[train_index], y[test_index]
        clf = GridSearchCV(svc, parameters)
        clf.fit(X_trn, y_trn)
        #print clf.best_estimator_
        acc = accuracy_score(clf.best_estimator_.predict(X_tst),y_tst)
        totalacc = totalacc + acc
        pred=clf.best_estimator_.predict(X_tst)
        cper.append(pred)
        ytt.append(y_tst)
        count = count + 1 
        #print count, acc, totalacc; # See the how many correct over the whole
    score=totalacc/(count-1)*100.0
    #print 'Accuracy of',a[cn],'=', totalacc/count
    print("Total accuracy score for {} is {}".format(a[cn], round(score,2)))
    #print (classification_report(np.array(ytt), np.array(cper)))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(np.array(ytt),np.array(cper))
    roc_acu=roc_auc = auc(false_positive_rate, true_positive_rate)
    print("ROC_AUC: {} is {}".format(a[cn], round(roc_acu,2)))
#     print '\n'
    cn+=1 

Total accuracy score for Theta is 41.94
ROC_AUC: Theta is 0.34
Total accuracy score for alpha is 51.61
ROC_AUC: alpha is 0.43
Total accuracy score for beta is 64.52
ROC_AUC: beta is 0.56
Total accuracy score for gamma is 45.16
ROC_AUC: gamma is 0.38


In [ ]:
# Total score for Theta is 59.375
# Total score for alpha is 62.5
# Total score for beta is 56.25
# Total score for gamma is 46.875

In [ ]:
#gamma (0.1,2,10)
# Total score for Theta is 46.875
# Total score for alpha is 40.625
# Total score for beta is 68.75
# Total score for gamma is 46.875
##parameters = {'kernel':('linear', 'rbf','poly','sigmoid'), 'C':[1, 10,100,1000],'gamma':[0.01,0.02,0.05,0.08,.002,0.003]}
# Total score for Theta is 61.29
# Total score for alpha is 64.52
# Total score for beta is 58.06
# Total score for gamma is 48.39

In [ ]:
# np.linspace(0.1,2,10)